# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sqlalchemy import create_engine

from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /home/jc2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jc2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jc2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jc2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table("messages", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df[df.related == 2]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,146,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
218,263,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
304,373,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
459,565,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
575,700,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20294,23662,transferred to a sanitary landfill site by a,None,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20465,23861,Families also have solar lamps which can be re...,None,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22292,25874,Read the [full blog post](http://www.odi.org.u...,None,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23346,27071,Actualmente e independientemente de la ayuda d...,None,news,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [4]:
X = df.message.values
Y = df.iloc[:, 4:]

In [5]:
df.message

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
                               ...                        
26172    The training demonstrated how to enhance micro...
26173    A suitable candidate has been selected and OCH...
26174    Proshika, operating in Cox's Bazar municipalit...
26175    Some 2,000 women protesting against the conduc...
26176    A radical shift in thinking came about as a re...
Name: message, Length: 26177, dtype: object

### 2. Write a tokenization function to process your text data

In [45]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t).lower().strip() for t in tokens]
    tokens = [t for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in set(stopwords.words('english'))]
    return tokens

In [46]:
X[:10]

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'says: west side of Haiti, rest of the country today and tonight',
       'Information about the National Palace-',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I would like to receive the messages, thank you',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'],
      dtype=object)

In [47]:
[tokenize(x) for x in X[:10]]

[['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'],
 ['hurricane'],
 ['looking', 'someone', 'name'],
 ['un',
  'report',
  'leogane',
  'destroyed',
  'hospital',
  'croix',
  'functioning',
  'needs',
  'supply',
  'desperately'],
 ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'],
 ['information', 'national'],
 ['storm', 'sacred', 'heart', 'jesus'],
 ['please', 'need', 'tent', 'water', 'silo', 'thank'],
 ['would', 'like', 'receive', 'message', 'thank'],
 ['health', 'issue', 'worker', 'santo', 'area']]

In [39]:
nltk.pos_tag(tokenize(X[3]))

[('un', 'JJ'),
 ('report', 'NN'),
 ('leogan', 'VBD'),
 ('destroy', 'JJ'),
 ('onli', 'NN'),
 ('hospit', 'NN'),
 ('croix', 'NN'),
 ('function', 'NN'),
 ('need', 'VBP'),
 ('suppli', 'NNS'),
 ('desper', 'VB')]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [28]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        # ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

In [40]:
for k in pipeline.get_params().keys():
    print(k)

memory
steps
verbose
features
clf
features__n_jobs
features__transformer_list
features__transformer_weights
features__verbose
features__text_pipeline
features__text_pipeline__memory
features__text_pipeline__steps
features__text_pipeline__verbose
features__text_pipeline__vect
features__text_pipeline__tfidf
features__text_pipeline__vect__analyzer
features__text_pipeline__vect__binary
features__text_pipeline__vect__decode_error
features__text_pipeline__vect__dtype
features__text_pipeline__vect__encoding
features__text_pipeline__vect__input
features__text_pipeline__vect__lowercase
features__text_pipeline__vect__max_df
features__text_pipeline__vect__max_features
features__text_pipeline__vect__min_df
features__text_pipeline__vect__ngram_range
features__text_pipeline__vect__preprocessor
features__text_pipeline__vect__stop_words
features__text_pipeline__vect__strip_accents
features__text_pipeline__vect__token_pattern
features__text_pipeline__vect__tokenizer
features__text_pipeline__vect__vocab

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [41]:
%%time
model = pipeline
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

CPU times: user 2min, sys: 40.7 s, total: 2min 40s
Wall time: 2min 42s


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [43]:
total_accuracy = []
for i in range(Y_test.shape[1]):
    c = classification_report(Y_test.iloc[:,i], Y_pred[:,i], output_dict=True, zero_division=0)
    total_accuracy.append(c.get("accuracy"))
sum(total_accuracy) / len(total_accuracy)

0.9095492742551566

In [25]:
for i in range(Y_test.shape[1]):
    print("=====", Y_test.columns[i] ,"=====")
    print(classification_report(Y_test.iloc[:,i], Y_pred[:,i], output_dict=False, zero_division=0))

===== related =====
              precision    recall  f1-score   support

           0       0.67      0.48      0.56      1497
           1       0.86      0.92      0.89      4996
           2       0.28      0.42      0.33        52

    accuracy                           0.82      6545
   macro avg       0.60      0.61      0.59      6545
weighted avg       0.81      0.82      0.81      6545

===== request =====
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5397
           1       0.84      0.48      0.62      1148

    accuracy                           0.89      6545
   macro avg       0.87      0.73      0.78      6545
weighted avg       0.89      0.89      0.88      6545

===== offer =====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6519
           1       0.00      0.00      0.00        26

    accuracy                           1.00      6545
   macro avg    

### 6. Improve your model
Use grid search to find better parameters. 

In [67]:
import importlib
import sys
importlib.reload(sys.modules['tets_cv'])
from tets_cv import build_model

[nltk_data] Downloading package punkt to /home/jc2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jc2/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jc2/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/jc2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
%%time
cv = build_model()
cv.fit(X_train, Y_train)
Y_pred = cv.predict(X_test)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
total_accuracy = []
for i in range(Y_test.shape[1]):
    c = classification_report(Y_test.iloc[:,i], Y_pred[:,i], output_dict=True, zero_division=0)
    total_accuracy.append(c.get("accuracy"))
sum(total_accuracy) / len(total_accuracy)

0.9473304473304474

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
cv.best_params_

{'features__text_pipeline__vect__ngram_range': (1, 2)}

### 9. Export your model as a pickle file

In [24]:
import pickle
with open('test_model.pkl', 'wb') as model_file:
    pickle.dump(cv, model_file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.